## Action items
1. spark read delta table from adls, use expression to group by for latestchanges.
2. call write api from adb to synapse sql dedicated pool to store latestchanges.
3. auth to sql pool using keyvault

In [159]:
val df_delta = spark.read.format("delta").load("abfss://wplushiramsynapsefs@wplushiramsynapseadlsv2.dfs.core.windows.net/synapse/workspaces/wplushiramsynapse/warehouse/lastchanges")
df_delta.show()

StatementMeta(threedot0, 89, 56, Finished, Available)

df_delta: org.apache.spark.sql.DataFrame = [key: string, time: bigint ... 2 more fields]
+---+----+--------+-------+
|key|time|newValue|deleted|
+---+----+--------+-------+
|  e|   7|      19|  false|
|  f|   8|      12|  false|
|  d|  10|      21|  false|
|  g|   9|    null|   true|
+---+----+--------+-------+



In [160]:
val latestonly = df_delta.selectExpr("key", "struct(time, newValue, deleted) as otherCols").groupBy("key").agg(max("otherCols").as("latest")).selectExpr("key", "latest.*")
latestonly.show()

StatementMeta(threedot0, 89, 57, Finished, Available)

latestonly: org.apache.spark.sql.DataFrame = [key: string, time: bigint ... 2 more fields]
+---+----+--------+-------+
|key|time|newValue|deleted|
+---+----+--------+-------+
|  g|   9|    null|   true|
|  f|   8|      12|  false|
|  e|   7|      19|  false|
|  d|  10|      21|  false|
+---+----+--------+-------+



In [161]:
latestonly.createOrReplaceTempView("latestonly")

StatementMeta(threedot0, 89, 58, Finished, Available)

In [162]:
%%sql
select * from latestonly

StatementMeta(threedot0, 89, 59, Finished, Available)

<Spark SQL result set with 4 rows and 4 fields>

https://stackoverflow.com/questions/26041479/query-sql-database-with-scala

In [185]:
Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")
import java.sql.{Connection, DriverManager, ResultSet, Statement}; 

def dt = {
    val connection_string = "jdbc:sqlserver://wplushiramsynapse.sql.azuresynapse.net:1433;database=wplussynapsedw;user=LoaderXL;password=XL!sqlPools2021"
    val connection = DriverManager.getConnection(connection_string)
    try {
        val statement = connection.createStatement()
        val results = statement.executeQuery(" if object_id('latestonly') is not null drop table latestonly; select 1")
    } finally {
        connection.close
    } 
}

dt

StatementMeta(threedot0, 89, 82, Finished, Available)

res254: Class[_] = class com.microsoft.sqlserver.jdbc.SQLServerDriver
import java.sql.{Connection, DriverManager, ResultSet, Statement}
dt: Unit


In [186]:
latestonly.write.option(Constants.SERVER, "wplushiramsynapse.sql.azuresynapse.net").synapsesql("wplussynapsedw.dbo.latestonly", Constants.INTERNAL)

StatementMeta(threedot0, 89, 83, Finished, Available)

## But he wants it in ADB.
1. https://docs.databricks.com/data/data-sources/azure/synapse-analytics.html#frequently-asked-questions-faq
2. https://github.com/Azure-Samples/azure-sql-db-databricks/blob/main/notebooks/03b-parallel-switch-in-load-into-partitioned-table-single.ipynb